# 入力を可変長にしたいのじゃ


GlobalAveragePooling2Dあるいは

SPP(Spatial Pyramid Pooling)使えば？とガチな人にアドバイスされ使ってみた
https://github.com/yhenon/keras-spp


参考

Variable-size image to convolutional layer

https://github.com/keras-team/keras/issues/1920

What is the advantage of using an InputLayer (or an Input) in a Keras model with Tensorflow tensors?

https://stackoverflow.com/questions/45217973/what-is-the-advantage-of-using-an-inputlayer-or-an-input-in-a-keras-model-with/45259874

How to give variable size images as input in keras

https://stackoverflow.com/questions/47795697/how-to-give-variable-size-images-as-input-in-keras



In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.1.6'

In [2]:
import keras.backend as K
print(K.image_dim_ordering())
print(K.image_data_format())


tf
channels_last


# 普通のサンプル

In [3]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, ZeroPadding2D
from keras.layers import Input, Dense, Dropout, Activation
from keras.models import Sequential
from keras.models import Model

model = Sequential()
model.add(Dense(32, activation='relu', input_dim=100))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# ダミーデータの作成
import numpy as np
data = np.random.random((1000, 100))
labels = np.random.randint(2, size=(1000, 1))

model.fit(data, labels, epochs=1, batch_size=32)

Epoch 1/1
1000/1000 [==============================] - 1s 1ms/step - loss: 0.7073 - acc: 0.4870


In [0]:
# from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
# from keras.layers import Input, Dense, Dropout, Activation
# from keras.models import Sequential
# from keras.models import Model


# model = Sequential()
# model.add(Dense(32, activation='relu', input_dim=(None)))
# model.add(Dense(1, activation='sigmoid'))
# model.compile(optimizer='rmsprop',
#               loss='binary_crossentropy',
#               metrics=['accuracy'])

# # ダミーデータの作成
# import numpy as np
# data = np.random.random((1000, 100))
# labels = np.random.randint(2, size=(1000, 1))

# model.fit(data, labels, epochs=1, batch_size=32)

In [0]:
# from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, ZeroPadding2D
# from sklearn.metrics import mean_squared_error
# def test():
#     tr = False
#     model = Sequential()
#     model.add(ZeroPadding2D((3,3),input_shape=(None,None,3), name='layer_0'))
#     model.add(Conv2D(64, (3, 3), activation='relu', trainable=tr))
#     model.add(ZeroPadding2D((1,1)))
#     model.add(Conv2D(64, (3, 3), activation='relu', trainable=tr))
#     model.add(MaxPooling2D((2,2), strides=(2,2)))
#     model.add(SpatialPyramidPooling([1,2,4]))
#     model.add(Dense(2, activation='softmax'))    
#     model.compile(loss=mean_squared_error, optimizer='adam')
#     return model

# t = test()
# t.summary()

# SpatialPyramidPooling

In [14]:
# !git clone https://github.com/yhenon/keras-spp.git
# !cp -R keras-spp/spp spp
# !ls spp

Cloning into 'keras-spp'...
remote: Counting objects: 125, done.
remote: Total 125 (delta 0), reused 0 (delta 0), pack-reused 125
Receiving objects: 100% (125/125), 29.25 KiB | 2.66 MiB/s, done.
Resolving deltas: 100% (76/76), done.


In [25]:
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, Activation, MaxPooling2D, Dense
from spp.SpatialPyramidPooling import SpatialPyramidPooling

batch_size = 64
num_channels = 3
num_classes = 10

model = Sequential()

# uses theano ordering. Note that we leave the image size as None to allow multiple image sizes
model.add(Conv2D(32, 3, 3, border_mode='same', input_shape=(None, None, 3)))
model.add(Activation('relu'))
model.add(Conv2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, 3, 3))
model.add(Activation('relu'))
model.add(SpatialPyramidPooling([1, 2, 4]))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='sgd')

# train on 64x64x3 images
model.fit(np.random.rand(batch_size, 64, 64, num_channels), np.zeros((batch_size, num_classes)))
# train on 32x32x3 images
model.fit(np.random.rand(batch_size, 32, 32, num_channels), np.zeros((batch_size, num_classes)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(None, Non..., padding="same")`
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`


Epoch 1/1
64/64 [==============================] - 2s 26ms/step - loss: 0.0000e+00
Epoch 1/1
64/64 [==============================] - 0s 2ms/step - loss: 0.0000e+00


In [21]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Input, Dense, Dropout, Activation
from keras.models import Model

nr_channels = 3
input_layer = Input(shape=(None, None, nr_channels))
#filters, kernel_size, strides=(1, 1),,,,
x = Conv2D(16,(4,4), activation = 'elu')(input_layer)  # single stride 4x4 filter for 16 maps
x = Conv2D(32,(4,4), activation = 'elu')(x)         # single stride 4x4 filter for 32 maps
x = Dropout(0.5)(x)
x = Conv2D(64,(4,4), activation = 'elu')(x)         # single stride 4x4 filter for 64 maps
x = Dropout(0.5)(x)
x = Conv2D(128, (1,1))(x)                           # finally 128 maps for global average-pool
x = GlobalAveragePooling2D()(x)                     # pseudo-dense 128 layer
output_layer = Dense(10, activation = "softmax")(x) # softmax output
model = Model(inputs = input_layer, outputs=output_layer)
model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics=["accuracy"])

x_train = np.random.random((1000, 4, 4, 1))
y_train = np.random.random((1000, 2))

model.fit(x_train, y_train, batch_size=200, nb_epoch=1, shuffle=False, verbose=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


ValueError: ignored

In [0]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import InputLayer

a = tf.placeholder(dtype=tf.float32, shape=(None, 784))

model = Sequential()
model.add(InputLayer(input_tensor=a, input_shape=(None, 784)))
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='softmax'))

output = model.output

In [0]:
# from keras.models import Sequential
# from keras.layers import LSTM, Dense
# import numpy as np
# from keras import backend as K

# model = Sequential()
# model.add(Dense(32, input_dim=(None)))
# model.add(Activation('relu'))

# x_train = np.random.random((1000, 50))
# y_train = np.random.random((1000, 2))

# model.fit(x_train, y_train, batch_size=200, nb_epoch=1, shuffle=False, verbose=1)

# Variable length sequence for RNN

LSTMはinput_shape=(None, 100)のように入力次元を可変にできる。

In [26]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np
from keras import backend as K

with K.get_session():

    # create model
    model = Sequential()
    model.add(LSTM(100, return_sequences=True, stateful=True, input_shape=(None, 100),
             batch_input_shape=(200, None, 100)))
    model.add(LSTM(100))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])

    # Generate dummy training data
    x_train = np.random.random((1000, 50, 100))
    x_train_2 = np.random.random((1000, 10, 100))
    y_train = np.random.random((1000, 2))
    y_train_2 = np.random.random((1000, 2))

    # Generate dummy validation data
    x_val = np.random.random((200, 50, 100))
    y_val = np.random.random((200, 2))

    # fit and eval models
    model.fit(x_train, y_train, batch_size=200, nb_epoch=1, shuffle=False, validation_data=(x_val, y_val), verbose=1)
    model.fit(x_train_2, y_train_2, batch_size=200, nb_epoch=1, shuffle=False, validation_data=(x_val, y_val), verbose=1)
    score = model.evaluate(x_val, y_val, batch_size=200, verbose=1)

RuntimeError: ignored

このNone手段を使用すると、バッチごとに異なるバッチサイズを設定できます。のfit_generator代わりに使用してくださいfit。1つのバッチは(1, 15, 100)、次のバッチは(1, 24, 100)どんなものでもよい。つまり、サイズ1のバッチを実行しているだけなので騒々しいですが、バッチごとに異なる長さのものを簡単に書くことができます。

1つのバッチ内で異なる長さが必要な場合は、マスキングを使用する必要があります。インバンドまたはアウトオブバンドで信号を送ることができます。帯域内は、ステップを無視する必要があることを示す特別な値になります。帯域外は1、含まれているステップと0除外されたステップのための独立した入力となります。あなたの損失にマスクを掛けるので、除外されたステップの損失は0になります。

上記の両方を行う場合、各バッチはそのバッチ内の最長シーケンスの長さになります。そのバッチ内のより短い配列は、パディングされマスクされる。これは、最も効率的な方法であり、毎回のデータ量が最小でなければなりません。

両方を行うのが最善でしょう。あなたが使用するならばfit、最長のシーケンスの長さまですべてをパディングする必要があります。本当に長い外れ値がある場合、それは大きな影響を与えます。

ここにはやや複雑な例がありますが、上記の方法の1つが含まれています。この実験は入力と出力を連結しているので、max(input len+output len)代わりに使用するだけmax(input)+max(output)ですが、それは概念実証であり、必要ではありません。
https://github.com/bstriner/keras-seq2seq

一番きれいなコードではありませんが、それがどのように動作するかを理解していれば、あなたは何でもできるはずです。

方法1および2は排他的ではない。1回だけ行うと、各バッチにシーケンスが1つしかないので、パディングは必要ありません。

異なるバッチ間の状態は、通常保持されていないか関連性がありません。statefulケラスのオプションは、あなたが何をしているかを確かめない限り、私がお勧めするものではありません。statefulバッチ間の状態は保持されますが、バッチ間では逆戻りしません。したがって、前の状態を使用しますが、前の状態を有効にすることを学ぶことはできません。

多くの人々statefulは、それが実際よりも多くのことを考えて問題にぶつかる。すべてのLSTMはステートフルでstatefulあり、バッチ間の状態のみであり、通常は悪い考えです。単一のバッチに収まらないような非常に長いシーケンスがある場合にのみ便利です。

学習はバッチ内でのみ機能します。20ステップ離れた依存関係を学習する必要がある場合は、各バッチに少なくとも20ステップが必要です。あなたは10の2ステートフルバッチを行うことができますが、それは有用な何かを学ぶかどうかはランダムです。

LSTMを訓練するための最も簡単で最良の方法は、多くの異なるサブシーケンスを取り、それらをすべてLSTMを通して実行することです（そうではありませんstateful）。そうすることで、シーケンス内のどこかで拾い上げることができるLSTMを学習しようとしていて、何か役に立つことができます。

LSTMの初期状態は、学習されたものです。その初期状態は、バッチ内の各シーケンスについて同じです。あなたのすべての事例で効果的な初期状態を学習します。

tldr; stateful正当な理由がない限り使用しないでください。それなしでトレーニングはより安定しています。

機械学習は、意味のある方法で一般化できるものを学ぶことです。それを行う最良の方法は、多くの異なるサブシーケンスとそれらのすべてで機能するLSTMを持つことです。より多くの方法で、データをより細かくスライスしてダイスすることができます。

極端な例として、茶色のコーパス全体を1つのバッチで1つのシーケンスとして訓練したLSTMを想像してみてください。それは学ぶだろうが、それは無意味だろう。コーパス内のどこからでも始まる可変長サブシーケンスを予測できるLSTMは、意味のある何かを学習することになります。

https://github.com/keras-team/keras/issues/6776


# 普通のCNN

mnistで動かす。

In [43]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 30
img_rows, img_cols = 28, 28
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train[:500]
y_train = y_train[:500]
x_test = x_test[:500]
y_test = y_test[:500]

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (500, 28, 28, 1)
500 train samples
500 test samples
Train on 500 samples, validate on 500 samples
Epoch 1/30
500/500 [==============================] - 1s 2ms/step - loss: 2.1072 - acc: 0.2640 - val_loss: 1.8231 - val_acc: 0.4780
Epoch 2/30
500/500 [==============================] - 0s 280us/step - loss: 1.5162 - acc: 0.5680 - val_loss: 2.2483 - val_acc: 0.1820
Epoch 3/30
500/500 [==============================] - 0s 291us/step - loss: 1.3621 - acc: 0.5740 - val_loss: 0.9003 - val_acc: 0.7000
Epoch 4/30
500/500 [==============================] - 0s 279us/step - loss: 0.9819 - acc: 0.6880 - val_loss: 0.8088 - val_acc: 0.7580
Epoch 5/30
500/500 [==============================] - 0s 267us/step - loss: 0.6503 - acc: 0.8200 - val_loss: 0.6590 - val_acc: 0.7720
Epoch 6/30
500/500 [==============================] - 0s 269us/step - loss: 0.5769 - acc: 0.8040 - val_loss: 0.5499 - val_acc: 0.8440
Epoch 7/30
500/500 [==============================] - 0s 274us/step - loss: 0.4789 - 

Epoch 22/30
500/500 [==============================] - 0s 258us/step - loss: 0.1202 - acc: 0.9600 - val_loss: 0.3503 - val_acc: 0.8960
Epoch 23/30
500/500 [==============================] - 0s 256us/step - loss: 0.0778 - acc: 0.9780 - val_loss: 0.3618 - val_acc: 0.8960
Epoch 24/30
500/500 [==============================] - 0s 268us/step - loss: 0.0682 - acc: 0.9720 - val_loss: 0.3436 - val_acc: 0.9060
Epoch 25/30
500/500 [==============================] - 0s 247us/step - loss: 0.0585 - acc: 0.9780 - val_loss: 0.3470 - val_acc: 0.9000
Epoch 26/30
500/500 [==============================] - 0s 245us/step - loss: 0.0530 - acc: 0.9880 - val_loss: 0.4279 - val_acc: 0.8860
Epoch 27/30
500/500 [==============================] - 0s 240us/step - loss: 0.0484 - acc: 0.9840 - val_loss: 0.3561 - val_acc: 0.9100
Epoch 28/30
500/500 [==============================] - 0s 257us/step - loss: 0.0525 - acc: 0.9820 - val_loss: 0.3668 - val_acc: 0.9040
Epoch 29/30
500/500 [==============================] - 

mnistをリサイズして可変の画像にする。

## 可変長入力可能のモデルで30 30のmnistにする。

In [27]:
from __future__ import print_function
import cv2
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 1
img_rows, img_cols = 28, 28
input_width, input_height = None, None
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train[:500]
y_train = y_train[:500]
x_test = x_test[:500]
y_test = y_test[:500]
#cv2で30 30にリサイズ
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
#     in_layer = Input(shape=(1, None, None))
    input_shape = (1, input_width, input_height)

else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
#     in_layer = Input(shape=(None, None,1))
    input_shape = (input_width, input_height, 1)

#28 28の画像から30 30に変換
x_train = np.array([cv2.resize(img[0] ,(30,30)).reshape(30,30,1) for img in x_train])
x_test = np.array([cv2.resize(img[0] ,(30,30)).reshape(30,30,1) for img in x_test])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), border_mode='same', activation='relu', input_shape=input_shape)) #input shape
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(GlobalAveragePooling2D())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (500, 30, 30, 1)
500 train samples
500 test samples


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(None, Non..., padding="same")`


Train on 500 samples, validate on 500 samples
Epoch 1/1
500/500 [==============================] - 1s 1ms/step - loss: 2.3025 - acc: 0.1200 - val_loss: 2.3022 - val_acc: 0.1340
Test loss: 2.3022018394470214
Test accuracy: 0.13400000005960463


## サイズを色々変えてみる

In [0]:
from __future__ import print_function
import cv2
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras import backend as K

def init_data(image_width_size, image_height_size):
        num_classes = 10
        img_rows, img_cols = 28, 28
        input_width, input_height = None, None
        (x_train, y_train), (x_test, y_test) = mnist.load_data()
        x_train = x_train[:500]
        y_train = y_train[:500]
        x_test = x_test[:500]
        y_test = y_test[:500]
        #cv2で30 30にリサイズ
        if K.image_data_format() == 'channels_first':
            x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
            x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
            input_shape = (1, input_width, input_height)

        else:
            x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
            x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
            input_shape = (input_width, input_height, 1)

        #28 28の画像から任意サイズに変換
        x_train = np.array([cv2.resize(img[0] ,(image_width_size,image_height_size)).reshape(image_width_size,image_height_size,1) for img in x_train])
        x_test = np.array([cv2.resize(img[0] ,(image_width_size,image_height_size)).reshape(image_width_size,image_height_size,1) for img in x_test])

        x_train = x_train.astype('float32')
        x_test = x_test.astype('float32')
        x_train /= 255
        x_test /= 255
        print('x_train shape:', x_train.shape)
        print(x_train.shape[0], 'train samples')
        print(x_test.shape[0], 'test samples')

        y_train = keras.utils.to_categorical(y_train, num_classes)
        y_test = keras.utils.to_categorical(y_test, num_classes)


def run_model(epochs = 100, batch_size = 128):  
        model = Sequential()
        model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape)) #input shape
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        model.add(GlobalAveragePooling2D())
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(num_classes, activation='softmax'))

        model.compile(loss=keras.losses.categorical_crossentropy,
                      optimizer=keras.optimizers.Adadelta(),
                      metrics=['accuracy'])

        model.fit(x_train, y_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  verbose=1,
                  validation_data=(x_test, y_test))
        score = model.evaluate(x_test, y_test, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

In [46]:
init_data(30,30)
run_model(epochs = 500)

x_train shape: (500, 30, 30, 1)
500 train samples
500 test samples
Train on 500 samples, validate on 500 samples
Epoch 1/500
500/500 [==============================] - 1s 2ms/step - loss: 2.3072 - acc: 0.1060 - val_loss: 2.3018 - val_acc: 0.1340
Epoch 2/500
500/500 [==============================] - 0s 237us/step - loss: 2.3017 - acc: 0.1320 - val_loss: 2.2990 - val_acc: 0.1440
Epoch 3/500
500/500 [==============================] - 0s 241us/step - loss: 2.2973 - acc: 0.1480 - val_loss: 2.2973 - val_acc: 0.1400
Epoch 4/500
500/500 [==============================] - 0s 231us/step - loss: 2.2933 - acc: 0.1920 - val_loss: 2.2952 - val_acc: 0.2200
Epoch 5/500
500/500 [==============================] - 0s 229us/step - loss: 2.2913 - acc: 0.1640 - val_loss: 2.2937 - val_acc: 0.2040
Epoch 6/500
500/500 [==============================] - 0s 225us/step - loss: 2.2849 - acc: 0.1800 - val_loss: 2.2923 - val_acc: 0.2080
Epoch 7/500
500/500 [==============================] - 0s 237us/step - loss: 2.

500/500 [==============================] - 0s 226us/step - loss: 2.1555 - acc: 0.2140 - val_loss: 2.1465 - val_acc: 0.1920
Epoch 27/500
500/500 [==============================] - 0s 222us/step - loss: 2.1221 - acc: 0.2020 - val_loss: 2.2585 - val_acc: 0.1480
Epoch 28/500
500/500 [==============================] - 0s 225us/step - loss: 2.1810 - acc: 0.1700 - val_loss: 2.0991 - val_acc: 0.2180
Epoch 29/500
500/500 [==============================] - 0s 215us/step - loss: 2.1423 - acc: 0.1980 - val_loss: 2.1784 - val_acc: 0.1760
Epoch 30/500
500/500 [==============================] - 0s 223us/step - loss: 2.1448 - acc: 0.1740 - val_loss: 2.1719 - val_acc: 0.1720
Epoch 31/500
500/500 [==============================] - 0s 217us/step - loss: 2.1257 - acc: 0.2200 - val_loss: 2.0907 - val_acc: 0.2140
Epoch 32/500
500/500 [==============================] - 0s 220us/step - loss: 2.1760 - acc: 0.1820 - val_loss: 2.1408 - val_acc: 0.1920
Epoch 33/500
500/500 [==============================] - 0s 22

500/500 [==============================] - 0s 227us/step - loss: 2.0092 - acc: 0.2460 - val_loss: 2.2812 - val_acc: 0.2020
Epoch 53/500
500/500 [==============================] - 0s 206us/step - loss: 1.9470 - acc: 0.3180 - val_loss: 1.8491 - val_acc: 0.3060
Epoch 54/500
500/500 [==============================] - 0s 218us/step - loss: 1.9073 - acc: 0.2700 - val_loss: 1.9471 - val_acc: 0.2800
Epoch 55/500
500/500 [==============================] - 0s 210us/step - loss: 1.9899 - acc: 0.2520 - val_loss: 1.8566 - val_acc: 0.3020
Epoch 56/500
500/500 [==============================] - 0s 210us/step - loss: 1.7720 - acc: 0.3680 - val_loss: 1.8216 - val_acc: 0.3220
Epoch 57/500
500/500 [==============================] - 0s 212us/step - loss: 1.8268 - acc: 0.3500 - val_loss: 2.0933 - val_acc: 0.2120
Epoch 58/500
500/500 [==============================] - 0s 226us/step - loss: 1.9401 - acc: 0.2560 - val_loss: 1.8300 - val_acc: 0.3260
Epoch 59/500
500/500 [==============================] - 0s 20

500/500 [==============================] - 0s 202us/step - loss: 1.8047 - acc: 0.3140 - val_loss: 1.8520 - val_acc: 0.2800
Epoch 79/500
500/500 [==============================] - 0s 209us/step - loss: 1.7936 - acc: 0.3000 - val_loss: 1.7263 - val_acc: 0.3300
Epoch 80/500
500/500 [==============================] - 0s 207us/step - loss: 1.6683 - acc: 0.3780 - val_loss: 1.7275 - val_acc: 0.3360
Epoch 81/500
500/500 [==============================] - 0s 211us/step - loss: 1.6376 - acc: 0.3780 - val_loss: 1.7079 - val_acc: 0.3260
Epoch 82/500
500/500 [==============================] - 0s 208us/step - loss: 1.5933 - acc: 0.4100 - val_loss: 1.7045 - val_acc: 0.3440
Epoch 83/500
500/500 [==============================] - 0s 207us/step - loss: 1.7030 - acc: 0.3580 - val_loss: 2.1298 - val_acc: 0.2240
Epoch 84/500
500/500 [==============================] - 0s 201us/step - loss: 1.9142 - acc: 0.2960 - val_loss: 1.7474 - val_acc: 0.3320
Epoch 85/500
500/500 [==============================] - 0s 21

500/500 [==============================] - 0s 208us/step - loss: 1.5226 - acc: 0.4320 - val_loss: 1.6214 - val_acc: 0.3820
Epoch 105/500
500/500 [==============================] - 0s 219us/step - loss: 1.4945 - acc: 0.4200 - val_loss: 1.7417 - val_acc: 0.3360
Epoch 106/500
500/500 [==============================] - 0s 201us/step - loss: 1.7598 - acc: 0.3360 - val_loss: 1.7105 - val_acc: 0.3480
Epoch 107/500
500/500 [==============================] - 0s 214us/step - loss: 1.5666 - acc: 0.4160 - val_loss: 1.6330 - val_acc: 0.3840
Epoch 108/500
500/500 [==============================] - 0s 198us/step - loss: 1.5417 - acc: 0.4240 - val_loss: 1.6477 - val_acc: 0.3640
Epoch 109/500
500/500 [==============================] - 0s 210us/step - loss: 1.5494 - acc: 0.4060 - val_loss: 1.7013 - val_acc: 0.3440
Epoch 110/500
500/500 [==============================] - 0s 198us/step - loss: 1.5904 - acc: 0.3780 - val_loss: 1.6791 - val_acc: 0.3400
Epoch 111/500
500/500 [==============================] 

...

500/500 [==============================] - 0s 213us/step - loss: 0.9307 - acc: 0.6820 - val_loss: 1.0975 - val_acc: 0.5760
Epoch 391/500
500/500 [==============================] - 0s 200us/step - loss: 1.0150 - acc: 0.6260 - val_loss: 1.2078 - val_acc: 0.5180
Epoch 392/500
500/500 [==============================] - 0s 211us/step - loss: 0.9510 - acc: 0.6600 - val_loss: 1.0764 - val_acc: 0.5780
Epoch 393/500
500/500 [==============================] - 0s 210us/step - loss: 0.9243 - acc: 0.6940 - val_loss: 1.0526 - val_acc: 0.6300
Epoch 394/500
500/500 [==============================] - 0s 223us/step - loss: 0.9256 - acc: 0.6900 - val_loss: 1.0402 - val_acc: 0.6220
Epoch 395/500
500/500 [==============================] - 0s 204us/step - loss: 0.8957 - acc: 0.6900 - val_loss: 1.0845 - val_acc: 0.5760
Epoch 396/500
500/500 [==============================] - 0s 228us/step - loss: 0.8920 - acc: 0.7040 - val_loss: 1.0647 - val_acc: 0.5880
Epoch 397/500
500/500 [==============================] 

500/500 [==============================] - 0s 207us/step - loss: 0.8421 - acc: 0.6980 - val_loss: 1.0347 - val_acc: 0.6400
Epoch 417/500
500/500 [==============================] - 0s 206us/step - loss: 0.9313 - acc: 0.6620 - val_loss: 1.0337 - val_acc: 0.6440
Epoch 418/500
500/500 [==============================] - 0s 202us/step - loss: 0.8880 - acc: 0.6800 - val_loss: 1.0000 - val_acc: 0.6320
Epoch 419/500
500/500 [==============================] - 0s 215us/step - loss: 0.8409 - acc: 0.7220 - val_loss: 1.0812 - val_acc: 0.5700
Epoch 420/500
500/500 [==============================] - 0s 196us/step - loss: 0.9424 - acc: 0.6900 - val_loss: 1.1182 - val_acc: 0.5560
Epoch 421/500
500/500 [==============================] - 0s 210us/step - loss: 0.8412 - acc: 0.7200 - val_loss: 0.9969 - val_acc: 0.6600
Epoch 422/500
500/500 [==============================] - 0s 198us/step - loss: 0.8441 - acc: 0.7040 - val_loss: 1.0358 - val_acc: 0.6340
Epoch 423/500
500/500 [==============================] 

Epoch 442/500
500/500 [==============================] - 0s 205us/step - loss: 0.8667 - acc: 0.6880 - val_loss: 1.0775 - val_acc: 0.5720
Epoch 443/500
500/500 [==============================] - 0s 216us/step - loss: 0.8411 - acc: 0.7440 - val_loss: 1.0076 - val_acc: 0.6140
Epoch 444/500
500/500 [==============================] - 0s 211us/step - loss: 0.8056 - acc: 0.7280 - val_loss: 1.0625 - val_acc: 0.6240
Epoch 445/500
500/500 [==============================] - 0s 203us/step - loss: 0.9014 - acc: 0.6920 - val_loss: 0.9697 - val_acc: 0.6560
Epoch 446/500
500/500 [==============================] - 0s 208us/step - loss: 0.8753 - acc: 0.7060 - val_loss: 0.9627 - val_acc: 0.6740
Epoch 447/500
500/500 [==============================] - 0s 209us/step - loss: 0.8305 - acc: 0.6960 - val_loss: 0.9936 - val_acc: 0.6280
Epoch 448/500
500/500 [==============================] - 0s 209us/step - loss: 0.7490 - acc: 0.7720 - val_loss: 0.9829 - val_acc: 0.6420
Epoch 449/500
500/500 [==================

500/500 [==============================] - 0s 219us/step - loss: 0.8519 - acc: 0.7060 - val_loss: 0.9944 - val_acc: 0.6200
Epoch 469/500
500/500 [==============================] - 0s 208us/step - loss: 0.7909 - acc: 0.7180 - val_loss: 1.0066 - val_acc: 0.6100
Epoch 470/500
500/500 [==============================] - 0s 220us/step - loss: 0.7797 - acc: 0.7420 - val_loss: 0.9334 - val_acc: 0.6660
Epoch 471/500
500/500 [==============================] - 0s 209us/step - loss: 0.7876 - acc: 0.7420 - val_loss: 0.9224 - val_acc: 0.6640
Epoch 472/500
500/500 [==============================] - 0s 221us/step - loss: 0.7760 - acc: 0.7440 - val_loss: 1.0297 - val_acc: 0.6040
Epoch 473/500
500/500 [==============================] - 0s 205us/step - loss: 0.8291 - acc: 0.7080 - val_loss: 0.9825 - val_acc: 0.6460
Epoch 474/500
500/500 [==============================] - 0s 221us/step - loss: 0.8953 - acc: 0.6600 - val_loss: 1.0678 - val_acc: 0.6200
Epoch 475/500
500/500 [==============================] 

Epoch 494/500
500/500 [==============================] - 0s 197us/step - loss: 0.7329 - acc: 0.7400 - val_loss: 0.9387 - val_acc: 0.6600
Epoch 495/500
500/500 [==============================] - 0s 202us/step - loss: 0.7747 - acc: 0.7400 - val_loss: 0.9131 - val_acc: 0.6840
Epoch 496/500
500/500 [==============================] - 0s 206us/step - loss: 0.7997 - acc: 0.7200 - val_loss: 0.9371 - val_acc: 0.6600
Epoch 497/500
500/500 [==============================] - 0s 202us/step - loss: 0.7280 - acc: 0.7480 - val_loss: 0.9135 - val_acc: 0.7000
Epoch 498/500
500/500 [==============================] - 0s 201us/step - loss: 0.7837 - acc: 0.7240 - val_loss: 0.9626 - val_acc: 0.6760
Epoch 499/500
500/500 [==============================] - 0s 212us/step - loss: 0.7820 - acc: 0.7300 - val_loss: 0.9073 - val_acc: 0.6840
Epoch 500/500
500/500 [==============================] - 0s 195us/step - loss: 0.6918 - acc: 0.7640 - val_loss: 0.9047 - val_acc: 0.6780
Test loss: 0.9047084989547729
Test accura

In [47]:
init_data(30,20)
run_model(epochs = 500)

x_train shape: (500, 30, 20, 1)
500 train samples
500 test samples
Train on 500 samples, validate on 500 samples
Epoch 1/500
500/500 [==============================] - 1s 2ms/step - loss: 2.2990 - acc: 0.1040 - val_loss: 2.2999 - val_acc: 0.0840
Epoch 2/500
500/500 [==============================] - 0s 239us/step - loss: 2.2948 - acc: 0.0860 - val_loss: 2.2977 - val_acc: 0.0860
Epoch 3/500
500/500 [==============================] - 0s 224us/step - loss: 2.2879 - acc: 0.1140 - val_loss: 2.2965 - val_acc: 0.0880
Epoch 4/500
500/500 [==============================] - 0s 203us/step - loss: 2.2867 - acc: 0.1080 - val_loss: 2.2944 - val_acc: 0.0940
Epoch 5/500
500/500 [==============================] - 0s 222us/step - loss: 2.2834 - acc: 0.1140 - val_loss: 2.2917 - val_acc: 0.1360
Epoch 6/500
500/500 [==============================] - 0s 224us/step - loss: 2.2774 - acc: 0.1280 - val_loss: 2.2886 - val_acc: 0.1740
Epoch 7/500
500/500 [==============================] - 0s 215us/step - loss: 2.

Epoch 26/500
500/500 [==============================] - 0s 217us/step - loss: 2.3308 - acc: 0.1660 - val_loss: 2.1109 - val_acc: 0.2340
Epoch 27/500
500/500 [==============================] - 0s 245us/step - loss: 2.0596 - acc: 0.2340 - val_loss: 2.1125 - val_acc: 0.2020
Epoch 28/500
500/500 [==============================] - 0s 240us/step - loss: 2.1460 - acc: 0.1680 - val_loss: 2.2966 - val_acc: 0.1460
Epoch 29/500
500/500 [==============================] - 0s 230us/step - loss: 2.1390 - acc: 0.1960 - val_loss: 2.0763 - val_acc: 0.2360
Epoch 30/500
500/500 [==============================] - 0s 229us/step - loss: 1.9917 - acc: 0.2800 - val_loss: 2.0804 - val_acc: 0.2220
Epoch 31/500
500/500 [==============================] - 0s 217us/step - loss: 2.1157 - acc: 0.2020 - val_loss: 2.2526 - val_acc: 0.2000
Epoch 32/500
500/500 [==============================] - 0s 216us/step - loss: 2.1473 - acc: 0.1900 - val_loss: 2.0486 - val_acc: 0.2560
Epoch 33/500
500/500 [==========================

500/500 [==============================] - 0s 209us/step - loss: 1.9178 - acc: 0.3120 - val_loss: 1.9432 - val_acc: 0.2560
Epoch 53/500
500/500 [==============================] - 0s 216us/step - loss: 1.8674 - acc: 0.3080 - val_loss: 1.9875 - val_acc: 0.2340
Epoch 54/500
500/500 [==============================] - 0s 212us/step - loss: 1.8215 - acc: 0.3180 - val_loss: 1.8625 - val_acc: 0.2860
Epoch 55/500
500/500 [==============================] - 0s 217us/step - loss: 1.8724 - acc: 0.2840 - val_loss: 2.1602 - val_acc: 0.2600
Epoch 56/500
500/500 [==============================] - 0s 210us/step - loss: 2.0474 - acc: 0.2340 - val_loss: 1.8864 - val_acc: 0.2940
Epoch 57/500
500/500 [==============================] - 0s 219us/step - loss: 1.8722 - acc: 0.3240 - val_loss: 1.8610 - val_acc: 0.3140
Epoch 58/500
500/500 [==============================] - 0s 208us/step - loss: 1.8049 - acc: 0.3300 - val_loss: 2.0246 - val_acc: 0.2360
Epoch 59/500
500/500 [==============================] - 0s 21

500/500 [==============================] - 0s 210us/step - loss: 1.8155 - acc: 0.3220 - val_loss: 1.7768 - val_acc: 0.3100
Epoch 79/500
500/500 [==============================] - 0s 230us/step - loss: 1.7587 - acc: 0.3380 - val_loss: 1.8330 - val_acc: 0.3020
Epoch 80/500
500/500 [==============================] - 0s 207us/step - loss: 1.6728 - acc: 0.3940 - val_loss: 1.7149 - val_acc: 0.3620
Epoch 81/500
500/500 [==============================] - 0s 214us/step - loss: 1.6792 - acc: 0.3880 - val_loss: 1.7241 - val_acc: 0.3240
Epoch 82/500
500/500 [==============================] - 0s 214us/step - loss: 1.6725 - acc: 0.3660 - val_loss: 1.8654 - val_acc: 0.2880
Epoch 83/500
500/500 [==============================] - 0s 211us/step - loss: 1.6294 - acc: 0.3540 - val_loss: 1.6983 - val_acc: 0.3520
Epoch 84/500
500/500 [==============================] - 0s 209us/step - loss: 1.7205 - acc: 0.3480 - val_loss: 1.7590 - val_acc: 0.3400
Epoch 85/500
500/500 [==============================] - 0s 21

500/500 [==============================] - 0s 226us/step - loss: 1.6033 - acc: 0.3800 - val_loss: 1.6937 - val_acc: 0.3560
Epoch 105/500
500/500 [==============================] - 0s 211us/step - loss: 1.5087 - acc: 0.4500 - val_loss: 1.6614 - val_acc: 0.3340
Epoch 106/500
500/500 [==============================] - 0s 226us/step - loss: 1.6390 - acc: 0.3820 - val_loss: 1.8775 - val_acc: 0.2740
Epoch 107/500
500/500 [==============================] - 0s 220us/step - loss: 1.7050 - acc: 0.3760 - val_loss: 1.6639 - val_acc: 0.3820
Epoch 108/500
500/500 [==============================] - 0s 211us/step - loss: 1.5048 - acc: 0.4360 - val_loss: 1.6282 - val_acc: 0.3860
Epoch 109/500
500/500 [==============================] - 0s 211us/step - loss: 1.5589 - acc: 0.4140 - val_loss: 1.7211 - val_acc: 0.3380
Epoch 110/500
500/500 [==============================] - 0s 222us/step - loss: 1.5274 - acc: 0.4220 - val_loss: 1.6179 - val_acc: 0.3740
Epoch 111/500
500/500 [==============================] 

...

500/500 [==============================] - 0s 218us/step - loss: 0.9140 - acc: 0.6880 - val_loss: 1.1670 - val_acc: 0.5780
Epoch 390/500
500/500 [==============================] - 0s 211us/step - loss: 1.0336 - acc: 0.6340 - val_loss: 1.0294 - val_acc: 0.6440
Epoch 391/500
500/500 [==============================] - 0s 214us/step - loss: 0.8808 - acc: 0.7060 - val_loss: 1.0198 - val_acc: 0.6320
Epoch 392/500
500/500 [==============================] - 0s 206us/step - loss: 0.8483 - acc: 0.7060 - val_loss: 1.0560 - val_acc: 0.5980
Epoch 393/500
500/500 [==============================] - 0s 207us/step - loss: 0.9133 - acc: 0.6660 - val_loss: 1.0213 - val_acc: 0.6440
Epoch 394/500
500/500 [==============================] - 0s 204us/step - loss: 0.9365 - acc: 0.6880 - val_loss: 1.1193 - val_acc: 0.5640
Epoch 395/500
500/500 [==============================] - 0s 216us/step - loss: 0.9704 - acc: 0.6820 - val_loss: 1.0938 - val_acc: 0.5840
Epoch 396/500
500/500 [==============================] 

500/500 [==============================] - 0s 211us/step - loss: 0.8537 - acc: 0.6940 - val_loss: 0.9848 - val_acc: 0.6400
Epoch 415/500
500/500 [==============================] - 0s 202us/step - loss: 0.8619 - acc: 0.7100 - val_loss: 0.9929 - val_acc: 0.6460
Epoch 416/500
500/500 [==============================] - 0s 214us/step - loss: 0.8824 - acc: 0.6960 - val_loss: 1.0224 - val_acc: 0.6180
Epoch 417/500
500/500 [==============================] - 0s 201us/step - loss: 0.8303 - acc: 0.7160 - val_loss: 0.9819 - val_acc: 0.6400
Epoch 418/500
500/500 [==============================] - 0s 210us/step - loss: 0.8166 - acc: 0.7180 - val_loss: 0.9649 - val_acc: 0.6600
Epoch 419/500
500/500 [==============================] - 0s 216us/step - loss: 0.8434 - acc: 0.7280 - val_loss: 1.1026 - val_acc: 0.5840
Epoch 420/500
500/500 [==============================] - 0s 208us/step - loss: 0.8589 - acc: 0.6720 - val_loss: 1.0625 - val_acc: 0.6000
Epoch 421/500
500/500 [==============================] 

500/500 [==============================] - 0s 207us/step - loss: 0.8332 - acc: 0.7300 - val_loss: 0.9917 - val_acc: 0.6420
Epoch 441/500
500/500 [==============================] - 0s 202us/step - loss: 0.8070 - acc: 0.7040 - val_loss: 0.9544 - val_acc: 0.6700
Epoch 442/500
500/500 [==============================] - 0s 216us/step - loss: 0.7929 - acc: 0.7300 - val_loss: 0.9785 - val_acc: 0.6300
Epoch 443/500
500/500 [==============================] - 0s 215us/step - loss: 0.8243 - acc: 0.6980 - val_loss: 0.9948 - val_acc: 0.6280
Epoch 444/500
500/500 [==============================] - 0s 207us/step - loss: 0.8499 - acc: 0.7180 - val_loss: 0.9568 - val_acc: 0.6760
Epoch 445/500
500/500 [==============================] - 0s 205us/step - loss: 0.8497 - acc: 0.7100 - val_loss: 0.9657 - val_acc: 0.6700
Epoch 446/500
500/500 [==============================] - 0s 196us/step - loss: 0.8026 - acc: 0.7420 - val_loss: 0.9732 - val_acc: 0.6540
Epoch 447/500
500/500 [==============================] 

Epoch 466/500
500/500 [==============================] - 0s 197us/step - loss: 0.9038 - acc: 0.6780 - val_loss: 1.0319 - val_acc: 0.6480
Epoch 467/500
500/500 [==============================] - 0s 220us/step - loss: 0.8390 - acc: 0.7060 - val_loss: 0.9549 - val_acc: 0.6620
Epoch 468/500
500/500 [==============================] - 0s 206us/step - loss: 0.7906 - acc: 0.7220 - val_loss: 0.9428 - val_acc: 0.6480
Epoch 469/500
500/500 [==============================] - 0s 213us/step - loss: 0.7389 - acc: 0.7720 - val_loss: 0.9588 - val_acc: 0.6600
Epoch 470/500
500/500 [==============================] - 0s 209us/step - loss: 0.7833 - acc: 0.7060 - val_loss: 0.9290 - val_acc: 0.6860
Epoch 471/500
500/500 [==============================] - 0s 207us/step - loss: 0.7622 - acc: 0.7300 - val_loss: 0.9520 - val_acc: 0.6620
Epoch 472/500
500/500 [==============================] - 0s 213us/step - loss: 0.8147 - acc: 0.7040 - val_loss: 1.0220 - val_acc: 0.6380
Epoch 473/500
500/500 [==================

500/500 [==============================] - 0s 204us/step - loss: 0.7889 - acc: 0.7320 - val_loss: 0.9086 - val_acc: 0.6780
Epoch 493/500
500/500 [==============================] - 0s 201us/step - loss: 0.7557 - acc: 0.7400 - val_loss: 0.9527 - val_acc: 0.6700
Epoch 494/500
500/500 [==============================] - 0s 217us/step - loss: 0.8356 - acc: 0.6720 - val_loss: 0.9048 - val_acc: 0.6760
Epoch 495/500
500/500 [==============================] - 0s 209us/step - loss: 0.7765 - acc: 0.7200 - val_loss: 0.9208 - val_acc: 0.6740
Epoch 496/500
500/500 [==============================] - 0s 223us/step - loss: 0.7207 - acc: 0.7540 - val_loss: 0.9104 - val_acc: 0.6800
Epoch 497/500
500/500 [==============================] - 0s 209us/step - loss: 0.7324 - acc: 0.7620 - val_loss: 0.9299 - val_acc: 0.6760
Epoch 498/500
500/500 [==============================] - 0s 211us/step - loss: 0.7781 - acc: 0.7400 - val_loss: 0.8990 - val_acc: 0.6820
Epoch 499/500
500/500 [==============================] 